In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
import re
from sklearn.model_selection import train_test_split
from skopt import gp_minimize
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [2]:
# Định nghĩa hàm filter_not_alphabet_or_number
def filter_not_alphabet_or_number(text):
    return re.sub('\W+', ' ', text)

In [3]:
# Tải dữ liệu từ file CSV vào DataFrame
data = pd.read_csv('data.csv')
questions = data['Question'].apply(filter_not_alphabet_or_number).values
labels = data['Label'].apply(filter_not_alphabet_or_number).values

In [4]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']

for question, label in zip(questions, labels):
    # Tokenize mỗi câu hỏi
    w = nltk.word_tokenize(question)
    words.extend(w)
    # Thêm câu hỏi và nhãn vào tập documents
    documents.append((w, label))

    # Thêm nhãn vào tập classes nếu chưa tồn tại
    if label not in classes:
        classes.append(label)

In [5]:
# Lemmatize và chuyển thành chữ thường các từ, loại bỏ các từ bị ignore
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# Sắp xếp tập classes
classes = sorted(list(set(classes)))

# Hiển thị thông tin về số câu hỏi, số lớp (nhãn) và từ vựng
print(len(questions), "câu hỏi")
print(len(classes), "lớp", classes)
print(len(words), "từ lemmatized duy nhất", words)

4121 câu hỏi
160 lớp ['1st_round_admission', '2nd_degree_accountant', '2nd_degree_english', '2nd_round_admission', '3rd_round_admission', 'accountant_majors', 'add_documents_later', 'additional_docs_after_applying_online', 'additional_required', 'admission_after_11_9', 'admission_application_time', 'admission_on_11_9', 'admission_options', 'admission_paper', 'admission_procedure_time', 'admission_records', 'apply_directly', 'apply_score_grade_11', 'apply_score_grade_12', 'apply_shool_profile_and_graduation_exam_score', 'apply_shool_profile_with_graduation_exam_score', 'area_field_in_registration_docs', 'ask_1st_batch_admission_list', 'ask_2nd_facility', 'ask_3rd_facility_training_car_majors', 'ask_3rd_facility_training_english_language_majors', 'ask_3rd_facility_training_mechanical', 'ask_4th_facility', 'ask_4x6_photo_mandatory', 'ask_Doan_book', 'ask_acronyms', 'ask_admission_conditions', 'ask_admission_procedure', 'ask_admission_when_graduating_from_previous_years', 'ask_admissions_o

In [6]:
# Lưu từ vựng và lớp (intents) vào các file pickle
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [7]:
# Tạo dữ liệu huấn luyện
training = []
# Tạo một mảng rỗng cho kết quả đầu ra
output_empty = [0] * len(classes)
# Biểu diễn bag-of-words cho mỗi câu hỏi
for doc in documents:
    # Khởi tạo bag-of-words
    bag = []
    # Danh sách các từ đã được phân tách (tokenize) cho mẫu câu hỏi
    pattern_words = doc[0]
    # Lemmatize mỗi từ - tạo từ cơ bản để đại diện cho các từ liên quan
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # Tạo mảng bag-of-words với giá trị 1 nếu từ hiện tại có trong mẫu câu hỏi, ngược lại giá trị 0
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # Kết quả đầu ra là '0' cho mỗi nhãn và '1' cho nhãn hiện tại (cho mỗi mẫu câu hỏi)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
    
# Xáo trộn các mẫu dữ liệu và chuyển thành mảng NumPy
random.shuffle(training)

# Thêm các phần tử 0 vào cuối mỗi mẫu để đảm bảo chúng có cùng kích thước
max_length = max(len(row[0]) for row in training)
print(max_length)
for row in training:
    if len(row[0]) < max_length:
        row[0] += [0] * (max_length - len(row[0]))

# Tạo các mảng riêng biệt cho biểu diễn bag-of-words và nhãn đầu ra
x = np.array([row[0] for row in training])
y = np.array([np.argmax(row[1]) for row in training])

1398


In [8]:
# Phân chia dữ liệu thành tập huấn luyện (train) và tập kiểm tra (test)
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=42)

In [278]:
# Số lượng lớp
num_classes = len(classes)
print(num_classes)

160


In [279]:
# Tạo mô hình - 3 lớp. Lớp đầu tiên có 128 neuron, lớp thứ hai có 64 neuron và lớp đầu ra có số neuron bằng số lượng nhãn (intents) để dự đoán nhãn đầu ra bằng softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [280]:
# Biên dịch mô hình. Sử dụng stochastic gradient descent với Nesterov accelerated gradient cho kết quả tốt cho mô hình này
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

C:\Users\baosp\miniconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [281]:
# # Tạo mô hình dưới dạng một hàm để sử dụng trong RandomizedSearchCV
# def create_model(epochs, batch_size):
#     model = Sequential()
#     model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(64, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(num_classes, activation='softmax'))

#     sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#     model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#     return model

# # Tạo wrapper cho mô hình Keras để sử dụng trong RandomizedSearchCV
# model = KerasClassifier(build_fn=create_model, verbose=0)

# # Xác định không gian tìm kiếm
# param_dist = {
#     'epochs': np.arange(10, 201),
#     'batch_size': np.arange(16, 129)
# }

# # Thực hiện Random Search
# random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=50, cv=3, verbose=1)
# random_search.fit(train_x, train_y)

# # Hiển thị thông số tối ưu
# print("Optimal epochs:", random_search.best_params_['epochs'])
# print("Optimal batch_size:", random_search.best_params_['batch_size'])
# print("Best accuracy:", random_search.best_score_)


In [282]:
# Huấn luyện mô hình
hist = model.fit(np.array(train_x), np.array(train_y), epochs=500, batch_size=32, verbose=1)

Epoch 1/500
103/103 [==============================] - 1s 3ms/step - loss: 5.0071 - accuracy: 0.0458
Epoch 2/500
103/103 [==============================] - 0s 3ms/step - loss: 4.7592 - accuracy: 0.0749
Epoch 3/500
103/103 [==============================] - 0s 3ms/step - loss: 4.5381 - accuracy: 0.0898
Epoch 4/500
103/103 [==============================] - 0s 3ms/step - loss: 4.3231 - accuracy: 0.1247
Epoch 5/500
103/103 [==============================] - 0s 3ms/step - loss: 4.1406 - accuracy: 0.1638
Epoch 6/500
103/103 [==============================] - 0s 3ms/step - loss: 3.9727 - accuracy: 0.1887
Epoch 7/500
103/103 [==============================] - 0s 3ms/step - loss: 3.8174 - accuracy: 0.2106
Epoch 8/500
103/103 [==============================] - 0s 3ms/step - loss: 3.6598 - accuracy: 0.2363
Epoch 9/500
103/103 [==============================] - 0s 3ms/step - loss: 3.5412 - accuracy: 0.2479
Epoch 10/500
103/103 [==============================] - 0s 3ms/step - loss: 3.3978 - accura

In [283]:
# Đánh giá mô hình trên tập kiểm tra
loss, accuracy = model.evaluate(test_x, test_y, verbose=0)
print(f"Loss on test data: {loss}")
print(f"Accuracy on test data: {accuracy}")

Loss on test data: 1.4960508346557617
Accuracy on test data: 0.7721211910247803


In [284]:
# lưu mô hình
model.save('chatbot.h5', hist)
print("model created")

model created


In [285]:
# Load the model
model = load_model('chatbot.h5')

# In ra tổng quan của mô hình
print(model.summary())

Model: "sequential_180"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_539 (Dense)           (None, 128)               179072    
                                                                 
 dropout_360 (Dropout)       (None, 128)               0         
                                                                 
 dense_540 (Dense)           (None, 64)                8256      
                                                                 
 dropout_361 (Dropout)       (None, 64)                0         
                                                                 
 dense_541 (Dense)           (None, 160)               10400     
                                                                 
Total params: 197,728
Trainable params: 197,728
Non-trainable params: 0
_________________________________________________________________
None


In [286]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
import pandas as pd

from keras.models import load_model
model = load_model('chatbot.h5')

import random

# Đọc dữ liệu từ file data.csv
data = pd.read_csv('data.csv')
questions = data['Question'].values
labels = data['Label'].values

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))


def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, data_df):
    tag = ints[0]['intent']
    responses = data_df[data_df['Label'] == tag]['Answer'].values
    result = random.choice(responses)
    return result
def chatbot_response(msg):
    ints = predict_class(msg, model)
    if ints:  # Kiểm tra xem danh sách ints có phần tử không
        res = getResponse(ints, data)
    else:
        res = "Xin lỗi, tôi không hiểu câu hỏi của bạn."
    return res


In [293]:
msg = "cs3 ở đâu"
chatbot_response(msg)

1/1 [==============================] - 0s 16ms/step


'Cơ sở hải dương: tại phường tân bình, thành phố hải dương, tỉnh hải dương với diện tích: 3.375 m2. cơ sở này cung cấp môi trường học tập cho khoảng 1.000 sv của các khoa: kinh tế, công nghệ may và thời trang, ngoại ngữ và sv hệ vừa làm vừa học (theo nhu cầu về địa điểm học tập của người học). Em tham khảo thêm mục 2. cơ sở vật chất: http://www.utehy.edu.vn/#/news-list/news-detail/73958047-4549-4216-8504-5a4446bc667d '